# Zillow for a Zipcode


This tool allows users to enter a zipcode, and see the following stats:
(1) Ave price of a property
(2) Ave no.beds of a property
(3) Ave no.baths of a property
(4) Ave price per beds
(5) Ave price per bath

In [161]:
## Import beautiful soup to scrape data from Zillow
!pip install beautifulsoup4
import requests
from bs4 import BeautifulSoup

In [162]:
## Import pandas
import pandas as pd

In [123]:
# Make a property list for a zipcode
property_list=[]
zipcode = raw_input(" Type a zipcode ")
count = 1
url = "https://www.zillow.com/homes/"+zipcode +"_rb/"+ str(count) +'_p/'
r = requests.get(url)
soup = BeautifulSoup(r.content) #make a soup for each url
prop_info = soup.find_all("div",{"class":"zsg-photo-card-caption"})

while soup.find_all("li", {'class':'zsg-pagination-next'}):
    #keep running the loop if there is a next bottom
    #Save all items in the new page to the property_list
    for item in prop_info:
        for i in item.find_all("span"):
            if i.get('class')==['zsg-photo-card-status']:
                try: 
                    status = i.text
                except:
                    status = 'NA'               
            if i.get('class')==['zsg-photo-card-price']:
                try:
                    price = float(i.text.replace("$","").replace("/mo","").replace(",","").replace("+",""))
                except:
                    price = 0.0
            if i.get('class')==['zsg-photo-card-info']:
                info = i.text.replace("Studio","0.5 bd")
                try:
                    bed = float(info.split()[0]) 
                except:
                    bed = 0.0
                try:
                    bath = float(info.split()[3])
                except:
                    bath = 0.0
                try:
                    size = float(info.split()[6].replace("--","0").replace("sqft","0").replace(",","").replace("+",""))
                except:
                    size = 0.0
            if i.get('class')==['zsg-photo-card-address']:
                try:
                    address = i.text
                except:
                    address = ''
        prop=[status,price,bed,bath,size,address]
        property_list.append(prop)
        
    #Make a new soup if the next page exists    
    count = count + 1
    url = "https://www.zillow.com/homes/"+zipcode +"_rb/"+ str(count) +'_p/'
    r = requests.get(url)
    soup = BeautifulSoup(r.content) #make a soup for each url
    prop_info = soup.find_all("div",{"class":"zsg-photo-card-caption"})

print ' There are ' + str(len(property_list)) + " properties listed in the zipcode of " + zipcode

 Type a zipcode 46033
 There are 202 properties listed in the zipcode of 46033


In [124]:
# Save the properties (in property_list) found in the zipcode to a pandas dataframe
labels = ['status','price','bed','bath','size','address']
df = pd.DataFrame.from_records(property_list, columns = labels)

In [125]:
df.head()

,status,price,bed,bath,size,address
0,For Sale by Owner,424900.0,5.0,3.0,3300.0,"11626 Forest Dr, Carmel, IN"
1,House For Sale,397750.0,4.0,3.0,4170.0,"14902 W Black Wolf Run Dr, Carmel, IN"
2,House For Sale,529900.0,5.0,4.0,6697.0,"12427 Pebble Knoll Way, Carmel, IN"
3,House For Rent,1800.0,4.0,2.5,2743.0,"11644 Victoria Ct, Carmel, IN"
4,House For Sale,849900.0,5.0,6.0,7348.0,"5041 Deer Ridge Ct, Carmel, IN"


In [126]:
df.describe()

,price,bed,bath,size
count,2.020000e+02,202.000000,202.000000,202.000000
mean,4.591700e+05,3.798020,3.492574,3783.153465
std,3.002416e+05,1.415356,1.595608,2051.690615
min,0.000000e+00,0.000000,0.000000,0.000000
25%,2.899248e+05,3.000000,3.000000,2448.000000
50%,4.359950e+05,4.000000,3.000000,3778.000000
75%,5.899000e+05,5.000000,4.000000,5105.500000
max,1.999999e+06,6.000000,8.000000,10860.000000


In [152]:
df_sale = df[df.status == "House For Sale"]

In [154]:
mean_bed = df_sale.describe()['bed']['mean']
print mean_bed

4.30952380952


In [156]:
mean_bath = df_sale.describe()['bath']['mean']
print mean_bath

4.02380952381


In [157]:
mean_size = df_sale.describe()['size']['mean']
print mean_size

4567.5952381


In [158]:
mean_price = int(df_sale.describe()['price']['mean'])
print mean_price

533938


In [159]:
mean_price_p_sqft = mean_price/mean_size
print mean_price_p_sqft

116.896960472


In [160]:
mean_price_p_b = mean_price/mean_bed
print mean_price_p_b

123897.21547


In [149]:
df.status.value_counts()

House For Sale               126
Pre-Foreclosure (Auction)     21
House For Rent                17
New Construction              16
For Sale by Owner              7
Make Me Move®                  4
Lot/Land For Sale              3
Condo For Rent                 2
Townhouse For Rent             2
Auction                        2
Woodland Springs Manor         1
Foreclosure                    1
Name: status, dtype: int64

In [153]:
df_sale.head()

,status,price,bed,bath,size,address
1,House For Sale,397750.0,4.0,3.0,4170.0,"14902 W Black Wolf Run Dr, Carmel, IN"
2,House For Sale,529900.0,5.0,4.0,6697.0,"12427 Pebble Knoll Way, Carmel, IN"
4,House For Sale,849900.0,5.0,6.0,7348.0,"5041 Deer Ridge Ct, Carmel, IN"
11,House For Sale,429900.0,3.0,3.0,4554.0,"14200 Arcadian Cir, Carmel, IN"
12,House For Sale,188000.0,3.0,3.0,1160.0,"11783 Esty Way, Carmel, IN"
